In [1]:
from model import *
from utils import *
import tensorflow as tf
from tqdm import tqdm, tnrange

In [2]:
batch_size = 50
z_dim = 100
data_dir = "../../data/"
data_name = "img_align_celeba"
print(len(glob.glob(os.path.join(data_dir+data_name + '/*.jpg'))))
train_iter,train_init,test_iter,test_init = data_feeder2([data_dir+data_name],"jpg",batch_size,0.1)

202599
tr: 182339  ts: 20260  tt: 202599


In [3]:
sess = tf.Session()
sess.run(train_init)
test_face_train_data = sess.run(train_iter.get_next())
print(test_face_train_data.shape)

(50, 218, 178, 3)


In [4]:
# logit = critic(test_face_train_data)

In [5]:
# sess.run(tf.global_variables_initializer())
# lg = sess.run(logit)

In [6]:
# print(lg.shape)

In [7]:
noise_dist = tf.contrib.distributions.Normal(0.,1.)
z = noise_dist.sample((batch_size, z_dim))

In [8]:
# gen4,gen3,gen2,gen1 = generator(z)

# sess.run(tf.global_variables_initializer())
# ge4,ge3,ge2,ge1 = sess.run([gen4,gen3,gen2,gen1])

In [9]:
# print(ge4.shape,ge3.shape,ge2.shape,ge1.shape)

In [10]:
def build_graph(gp=False):
    noise_dist = tf.contrib.distributions.Normal(0.,1.)
    z = noise_dist.sample((batch_size, z_dim))
    with tf.variable_scope('generator'):
        gen_data = generator(z)
    real_data = tf.placeholder(
        dtype=tf.float32,
        shape=((batch_size,218,178,3)))
    true_logit = critic(real_data)
    fake_logit = critic(gen_data, reuse=True)
    c_loss = tf.reduce_mean(fake_logit-true_logit)
    if gp == True:
        gp = False
    g_loss = tf.reduce_mean(-fake_logit)
    
    g_loss_sum = tf.summary.scalar("g_loss", g_loss)
    c_loss_sum = tf.summary.scalar("c_loss", c_loss)
    img_sum = tf.summary.image("gen", gen_data, max_outputs=10)
    theta_g = tf.get_collection(
        tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
    theta_c = tf.get_collection(
        tf.GraphKeys.TRAINABLE_VARIABLES, scope='critic')
    opt_g = tf.train.RMSPropOptimizer(1e-3).minimize(g_loss, var_list = theta_g)
    opt_c = tf.train.RMSPropOptimizer(1e-3).minimize(c_loss, var_list = theta_c)
    if gp==False:
        clipped_var_c = [tf.assign(var, tf.clip_by_value(var, -0.01, 0.01)) for var in theta_c]
        # merge the clip operations on critic variables
        with tf.control_dependencies([opt_c]):
            opt_c = tf.tuple(clipped_var_c)
    return opt_g,opt_c,real_data        

In [11]:
def train(log_dir = "logs",epoch = 100,Citers=5):
    opt_g, opt_c, real_data = build_graph()
    saver = tf.train.Saver()
    merged_all = tf.summary.merge_all()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
        t1 = tnrange(epoch,desc = '1st loop')
        t2 = tnrange(int(252559/batch_size),desc = '2nd loop',leave= False)
        for e in t1:
            sess.run(train_init)
            next_ele = train_iter.get_next()
            for step in t2:
                i = e*int(252559/batch_size)+step
                next_batch = sess.run(next_ele)
                if i < 25 or i % 500 == 0:
                    citers = 20
                else:
                    citers = Citers
                for j in range(citers):
                    next_batch = sess.run(next_ele)
                    if i % 10 == 9 and j == 0:
                        run_metadata = tf.RunMetadata()
                        _, merged = sess.run([opt_c, merged_all], feed_dict={real_data:next_batch})
                        summary_writer.add_summary(merged, i)
                        summary_writer.add_run_metadata(
                            run_metadata, 'critic_metadata {}'.format(i), i)
                    else:
                        sess.run(opt_c, feed_dict={real_data:next_batch})
                next_batch = sess.run(next_ele)
                if i % 10 == 9:
                    _, merged = sess.run([opt_g, merged_all], feed_dict={real_data:next_batch})
                    summary_writer.add_summary(merged, i)
                    summary_writer.add_run_metadata(
                        run_metadata, 'generator_metadata {}'.format(i), i)
                else:
                    sess.run(opt_g, feed_dict={real_data:next_batch})                
                if i % 1000 == 999:
                    saver.save(sess, os.path.join(
                        ckpt_dir, "model.ckpt"), global_step=i)

In [12]:
train()

KeyboardInterrupt: 